In [1]:
import json
import requests, re
from bs4 import BeautifulSoup
import pandas as pd
import pprint
import mysql.connector
import time 
import os
import html.parser as htmlparser
import numpy as np
import pymysql

# Part 1:  SHORT, LET GET STARTED

Go to http://numbersapi.com and familiarize yourself with the API.

a) Write a program that accesses all trivia results for the numbers from 0 (zero) to 99 using batch requests only (One single query for all the numbers). Print the output result to the screen in the format of [3-digit number with leading zeros] - [TRIVIA] (One number per each line). E.g.,

In [2]:
#############################################################
#Part 1
#############################################################

#set up the api url
url_base = 'http://numbersapi.com/'
query_type = 'trivia'
batch = '0..99'
api = url_base + batch + '/' + query_type

#connect to the api url
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36'
}
data = requests.get(api,headers = headers)

#parse the result in the json format
results = json.loads(data.content.decode('utf-8'))

#iteration for printing the result
for i in range(0,100):
    index = '{0:03}'.format(i)
    print (index+' - '+results[str(i)])


000 - 0 is the coldest possible temperature old the Kelvin scale.
001 - 1 is the number of Gods in monotheism.
002 - 2 is the number of stars in a binary star system (a stellar system consisting of two stars orbiting around their center of mass).
003 - 3 is cans of Spam consumed every second in the United States.
004 - 4 is the number of bits in a nibble, equivalent to half a byte.
005 - 5 is the number of completed, numbered piano concertos of Ludwig van Beethoven, Sergei Prokofiev, and Camille Saint-Saëns.
006 - 6 is the number of strings on a standard guitar.
007 - 7 is the number of days in a week.
008 - 8 is the number of bits in a byte.
009 - 9 is the number of innings in a regulation, non-tied game of baseball.
010 - 10 is the number of years in a decade.
011 - 11 is the possible age of the youngest elected pope, Benedict IX.
012 - 12 is the number of ounces in a troy pound (used for precious metals).
013 - 13 is the speed of rush hour traffic on average in kilometres per hour i

# Part 2: LONGER!

Web-scraping: Which sellers advertise/sponsor on eBay

a) Take the code your group developed for assignment 2 and fix everything that may not have been optimal in your previous solution. Alternatively, but less cool, you may use our solution code as your starting point. (You will use this for the next part. You do not need to include this code in your submission.)

How would you modify your search query to only include buy-it-now (non-auction) items? What is the GET request's variable name corresponding to buy-it-now searches? How would you modify your search query to include 100 items per search result page? What is the GET request's variable name corresponding to items per page searches? Include your answers as a comment in your code marking it as Part 2-a. (This section does not involve coding)

In [3]:
#############################################################
#Part 2
#Note: If item's return policy is NUll, the item was sold out.
#############################################################

#a

# I would modify the search url to this: https://www.ebay.com/sch/i.html?_nkw=sumsung+tv&LH_BIN=1&_ipg=100. While still using 
#_nkw for the searching item, I added two GET request's variable for buy-it-now items and 100 items per page.
# The LH_BIN is the variable name that let the search to filter the search results to only buy-it-now items when setting 
#the variable equal to 1.
#The _ipg is the variable name that let the search to display results as 100 items per page. 


b) Modify your previous code to search for buy-it-now listings of "ipad 4" and limit the number of items to 100 per page.

Identify all the sponsored items.

For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

In [76]:
#b

#Set up a base link and header
ebaylink='https://www.ebay.com/sch/i.html?_nkw=ipad+4&LH_BIN=1&_ipg=100&_pgn='
headers = {
    'User-Agent': 'Mozilla/5.0 '
}

#create files
sp_text = open('sponsored.txt', "w",encoding='utf-8') 
nsp_text = open('non-sponsored.txt', "w",encoding='utf-8')

#iteration for pages
for i in range(1,11):
    
    #create the url and connect
    url = ebaylink+str(i)
    data = requests.get(url,headers = headers)
    soup = BeautifulSoup(data.text, 'html.parser')

    #locating item elements
    items = soup.select("div#srp-river-main > div#srp-river-results > ul > li.s-item")
    s_n = 0
    n_n = 0
    #iteration for each item
    for item in items:
        
        #extract the item links
        itemlink = item.find('a','s-item__link')['href']
        
        #detect SPONSORED
        sponsored_tag = item.find('span',{'role':'text'})
        #test whether the tag displayed as "SPONSORED" and write the link to files
        if sponsored_tag != None:
            text = item.find('span',{'role':'text'}).get_text()
            if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text):
                sp_text.write(str(itemlink)+'\n')
                s_n = s_n+1
            else:
                nsp_text.write(str(itemlink)+'\n')
                n_n = n_n+1  
        else:
            nsp_text.write(str(itemlink)+'\n')
            n_n = n_n+1
    
    print('Page: ',i,' Sponsored: ',s_n,' Non-Sponsored: ',n_n)
    time.sleep(1)

#close files
sp_text.close()
nsp_text.close()


Page:  1  Sponsored:  13  Non-Sponsored:  100
Page:  2  Sponsored:  13  Non-Sponsored:  100
Page:  3  Sponsored:  13  Non-Sponsored:  100
Page:  4  Sponsored:  13  Non-Sponsored:  100
Page:  5  Sponsored:  13  Non-Sponsored:  100
Page:  6  Sponsored:  13  Non-Sponsored:  100
Page:  7  Sponsored:  13  Non-Sponsored:  100
Page:  8  Sponsored:  13  Non-Sponsored:  100
Page:  9  Sponsored:  13  Non-Sponsored:  100
Page:  10  Sponsored:  13  Non-Sponsored:  100


c) Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "<item-id>.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [77]:
#c

######SPONSORED################

#create a folder called sponsored
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'sponsored')
if not os.path.exists(final_directory):
    os.makedirs(final_directory)

#read the links
file = open("sponsored.txt", "r")
f = file.readlines()

#iteration for downloading web pages
error_link = list()
success_num = 0

for i in range(len(f)):
    
    ebaylink=f[i]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36'
    }
    
    try:
        #connect the page
        data = requests.get(ebaylink,headers = headers)
        soup = BeautifulSoup(data.text, 'html.parser')

        #find the item id
        ID = soup.find('div',attrs={'id':'descItemNumber'}).text.strip()
        filename = ID + '.htm'
        
        #set the path and output the page as htm files
        full_filename = os.path.join(final_directory, filename)
        with open(full_filename, "w",encoding='utf-8') as file:
            file.write(str(soup))
            file.close()
            success_num = success_num + 1  #count the pages
    except:
        
        #error control 
        error_link = error_link.append(f[i])
        print('Error Message: ',sys.exc_info()[0])
        continue
    
    time.sleep(2)

print('===============Sponsored==================')
print('Total links: ',len(f),'. \n Success Downloaded: ',success_num,'. \n Error Links: ',error_link)

===============Sponsored==================
Total links:  130 . 
 Success Downloaded:  130 . 
 Error Links:  []


In [78]:

######NON_SPONSORED################

#create a folder called sponsored
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'non-sponsored')
if not os.path.exists(final_directory):
    os.makedirs(final_directory)

#read the links
file = open("non-sponsored.txt", "r")
f = file.readlines()

#iteration for downloading web pages
error_link = list()
success_num = 0

for i in range(len(f)):
    
    ebaylink=f[i]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36'
    }
    
    try:
        #connect the page
        data = requests.get(ebaylink,headers = headers)
        soup = BeautifulSoup(data.text, 'html.parser')

        #find the item id
        ID = soup.find('div',attrs={'id':'descItemNumber'}).text.strip()
        filename = ID + '.htm'
        
        #set the path and output the page as htm files
        full_filename = os.path.join(final_directory, filename)
        with open(full_filename, "w",encoding='utf-8') as file:
            file.write(str(soup))
            file.close()
            success_num = success_num + 1  #count the pages
    except:
        
        #error control 
        error_link = error_link.append(f[i])
        print('Error Message: ',sys.exc_info()[0])
        continue
    
    time.sleep(2)

print('===============Non Sponsored===================')
print('Total links: ',len(f),'. \n Success Downloaded: ',success_num,'. \n Error Links: ',error_link)

===============Non Sponsored===================
Total links:  1000 . 
 Success Downloaded:  1000 . 
 Error Links:  []


d) Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In your code, highlight the selector command you choose to obtain each element using comments.

In [97]:
#d

###########Sponsored###############################
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'sponsored')
ebay_info = []
for filename in os.listdir(final_directory):
    path = final_directory+'\\'+filename

    with open(path,encoding="utf-8") as page:
        #parse as beautifulsoup object
        soup=BeautifulSoup(page, 'html.parser')

        #Item ID
        item_id = filename.replace('.htm','')
        #Seller Name
        seller_name = [tag.text.strip() for tag in soup.select('span.mbg-nw')] #select element span with class mbg-nw 
        #Seller Score
        seller_score = [tag.text.strip() for tag in soup.select('span.mbg-l > a')] #select element a under element span with class mbg-l
        if len(seller_score)>1:
            seller_score = seller_score[0]
        else:
            seller_score = seller_score
        #Item Price
        item_price = [tag.text.strip() for tag in soup.select("span[itemprop='price']")] #select element span with attribute itemprop = price
        if len(item_price)>1:
            item_price = item_price[0]
        elif len(item_price)==0:
            item_price = [tag.text.strip() for tag in soup.select("span#prcIsum")] #if the item was sold select element span with id prcIsum
        else:
            item_price = item_price
        #Number of items sold
        item_sold = [tag.text.strip() for tag in soup.select('a.vi-txt-underline')] #select element a with class vi-txt-underline
        if item_sold !=[]:
            item_sold = item_sold
        else:
            item_sold = '0'
        #Best Offer Available
        best_offer = [tag.text.strip() for tag in soup.select('a#boBtn_btn')] #select element a with id boBtn_btn
        if len(best_offer) !=0:
            best_offer = '1'
        else:
            best_offer = '0'
        #Title
        item_title = [tag.text.strip() for tag in soup.select('h1#itemTitle')] #select element h1 with id itemTitle
        #Return Allowed
        return_policy = [tag.text.strip() for tag in soup.select('span#vi-ret-accrd-txt')] #select element span with id vi-ret-accrd-txt
        if len(return_policy) == 0:
            return_policy = None
        else:
            return_policy = return_policy
        #Shiping Price
        shiping_price = [tag.text.strip() for tag in soup.select('span#shSummary')] #select element span with id shSummary
        #Condition
        item_condition = [tag.text.strip() for tag in soup.select('div#vi-itm-cond')] #select element div with id vi-itm-cond
        
        ebay_data = [{
                  'item_id': item_id,
                  'seller_name':seller_name,
                  'seller_score':seller_score,
                  'item_price':item_price,
                  'item_sold':item_sold,
                  'best_offer': best_offer,
                  'item_title': item_title,
                  'return_policy': return_policy,
                  'shiping_price': shiping_price,
                  'item_condition': str(item_condition),
                  'item_type': 'sponsored'
                  }
                ]

        ebay_info.extend(ebay_data)
    sponsored =  pd.DataFrame(ebay_info)
    


In [98]:
###########Non-Sponsored###############################
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, r'non-sponsored')
ebay_info = []
for filename in os.listdir(final_directory):
    path = final_directory+'\\'+filename

    with open(path,encoding="utf-8") as page:
        #parse as beautifulsoup object
        soup=BeautifulSoup(page, 'html.parser')

        #Item ID
        item_id = filename.replace('.htm','')
        #Seller Name
        seller_name = [tag.text.strip() for tag in soup.select('span.mbg-nw')] #select element span with class mbg-nw 
        #Seller Score
        seller_score = [tag.text.strip() for tag in soup.select('span.mbg-l > a')] #select element a under element span with class mbg-l
        if len(seller_score)>1:
            seller_score = seller_score[0]
        else:
            seller_score = seller_score
        #Item Price
        item_price = [tag.text.strip() for tag in soup.select("span[itemprop='price']")] #select element span with attribute itemprop = price
        if len(item_price)>1:
            item_price = item_price[0]
        elif len(item_price)==0:
            item_price = [tag.text.strip() for tag in soup.select("span#prcIsum")] #if the item was sold select element span with id prcIsum
        else:
            item_price = item_price
        #Number of items sold
        item_sold = [tag.text.strip() for tag in soup.select('a.vi-txt-underline')] #select element a with class vi-txt-underline
        if item_sold !=[]:
            item_sold = item_sold
        else:
            item_sold = '0'
        #Best Offer Available
        best_offer = [tag.text.strip() for tag in soup.select('a#boBtn_btn')] #select element a with id boBtn_btn
        if len(best_offer) !=0:
            best_offer = '1'
        else:
            best_offer = '0'
        #Title
        item_title = [tag.text.strip() for tag in soup.select('h1#itemTitle')] #select element h1 with id itemTitle
        #Return Allowed
        return_policy = [tag.text.strip() for tag in soup.select('span#vi-ret-accrd-txt')] #select element span with id vi-ret-accrd-txt
        if len(return_policy) == 0:
            return_policy = None
        else:
            return_policy = return_policy
        #Shiping Price
        shiping_price = [tag.text.strip() for tag in soup.select('span#shSummary')] #select element span with id shSummary
        #Condition
        item_condition = [tag.text.strip() for tag in soup.select('div#vi-itm-cond')] #select element div with id vi-itm-cond
        
        ebay_data = [{
                  'item_id': item_id,
                  'seller_name':seller_name,
                  'seller_score':seller_score,
                  'item_price':item_price,
                  'item_sold':item_sold,
                  'best_offer': best_offer,
                  'item_title': item_title,
                  'return_policy': return_policy,
                  'shiping_price': shiping_price,
                  'item_condition': str(item_condition),
                  'item_type': 'non-sponsored'
                  }
                ]

        ebay_info.extend(ebay_data)
    nonsponsored =  pd.DataFrame(ebay_info)
    


e) Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (d)  into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert $12.34 into 1234 and $12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [100]:
#e

###########Dataframe Cleaning and Combination##############

ebay_df = pd.concat([sponsored,nonsponsored])


ebay_df.item_condition=ebay_df.item_condition.apply(lambda x:str(x).strip("['']"))
ebay_df.item_price=ebay_df.item_price.apply(lambda x:str(x).strip("['']"))
ebay_df.item_price=ebay_df.item_price.apply(lambda x:str(x).replace(",",''))
ebay_df.item_sold=ebay_df.item_sold.apply(lambda x:str(x).strip("['']"))
ebay_df.item_title=ebay_df.item_title.apply(lambda x:str(x).strip("['']"))
ebay_df.return_policy=ebay_df.return_policy.apply(lambda x:str(x).strip("['']"))
ebay_df.seller_name=ebay_df.seller_name.apply(lambda x:str(x).strip("['']"))
ebay_df.seller_score=ebay_df.seller_score.apply(lambda x:str(x).strip("['']"))
ebay_df.shiping_price=ebay_df.shiping_price.apply(lambda x:str(x).strip("[]"))

#Transform item price
ebay_df['item_price_unit']=ebay_df.item_price.apply(lambda x:re.findall(r'[A-Z]+',str(x)))
ebay_df.item_price=ebay_df.item_price.apply(lambda x:re.findall(r'\d+.\d*',str(x)))
ebay_df.item_price=ebay_df.item_price.apply(lambda x:str(x).strip("['']"))

ebay_df.item_price_unit=ebay_df.item_price_unit.apply(lambda x:str(x).strip("['']"))
for item in range(0,len(ebay_df.item_price)):
    try:
        ebay_df.item_price.iloc[item] = "{:.2f}".format(float(ebay_df.item_price.iloc[item]))
        ebay_df.item_price.iloc[item] = str(int(float(ebay_df.item_price.iloc[item])*100))
    except:
        ebay_df.item_price.iloc[item] = None
        continue

#Transform item sold
for item in range(0,len(ebay_df.best_offer)):
    if not ebay_df.item_sold.iloc[item]:
        ebay_df.item_sold.iloc[item] = None
    else:
        ebay_df.item_sold.iloc[item] = str(int(re.findall(r'\d+',str(ebay_df.item_sold.iloc[item]))[0]))
        
#Transform item title
ebay_df.item_title=ebay_df.item_title.apply(lambda x:str(x).replace("\\xa0",''))

#Transform item title
ebay_df.return_policy=ebay_df.return_policy.apply(lambda x:str(x).replace("\\xa0",''))

#Transform shiping_price
ebay_df.shiping_price=ebay_df.shiping_price.apply(lambda x:re.findall(r'FREE|Free|Calculate|\$\d+.\d*\\',x))
def shiping_first(txt):
    if len(txt)>1:
        return txt[0]
    else:
        return txt
    
ebay_df.shiping_price=ebay_df.shiping_price.apply(lambda x:shiping_first(x))

ebay_df.shiping_price=ebay_df.shiping_price.apply(lambda x:str(x).strip("['']"))

ebay_df['shiping_price']=np.where(ebay_df['shiping_price'] =='FREE', "0", ebay_df['shiping_price'])
ebay_df['shiping_price']=np.where(ebay_df['shiping_price'] =='Free', "0", ebay_df['shiping_price'])
ebay_df['shiping_price_compute']=np.where(ebay_df['shiping_price'] =='Calculate', "1","0")
ebay_df['shiping_price']=np.where(ebay_df['shiping_price'] =='Calculate', '', ebay_df['shiping_price'])
ebay_df.shiping_price=ebay_df.shiping_price.apply(lambda x:str(x).replace("$",''))
ebay_df.shiping_price=ebay_df.shiping_price.apply(lambda x:str(x).replace('\\',''))

for item in range(0,len(ebay_df.best_offer)):
    try:
        ebay_df.shiping_price.iloc[item] = str(int(float(ebay_df.shiping_price.iloc[item])*100))
    except:
        ebay_df.shiping_price.iloc[item] = None
        continue
        
#Transform seller score
for item in range(0,len(ebay_df.best_offer)):
    try:
        ebay_df.seller_score.iloc[item] = str(ebay_df.seller_score.iloc[item])
    except:
        ebay_df.seller_score.iloc[item] = None
        continue

In [101]:
##############Save to Database######################################
# Connect to mysql
mydb = mysql.connector.connect(
  host="localhost",
  user="root"
)

mycursor = mydb.cursor()
print(mydb)


In [102]:
# Create database
mycursor.execute( "Create DATABASE if not exists eBay")

In [103]:
#Create table 

# if the table already existed, run following code
# mydb.close()
# mydb = mysql.connector.connect(
#   host="localhost",
#   user="root",
#   database='eBay'
# )
# mycursor = mydb.cursor()
# mycursor.execute( "DROP TABLE eBay_items")
mydb.close()

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  database='eBay'
)
mycursor = mydb.cursor()

mycursor.execute("CREATE TABLE eBay_items (item_id VARCHAR(255)   ,best_offer VARCHAR(255),\
            item_condition VARCHAR(255), item_price INT(255),item_price_unit VARCHAR(10),\
            item_sold INT(255), item_title VARCHAR(255), item_type VARCHAR(255),\
            return_policy VARCHAR(255), seller_name VARCHAR(255),seller_score VARCHAR(255),\
            shiping_price INT(255), shiping_price_compute BOOL)")

In [104]:
# Insert data
for i in range(len(ebay_df.best_offer)):
    sql = "INSERT INTO eBay_items (item_id,best_offer, item_condition,item_price,item_price_unit,\
            item_sold,item_title,item_type,return_policy,seller_name,seller_score,\
            shiping_price,shiping_price_compute) VALUES (%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    val = [(ebay_df['item_id'].iloc[i],ebay_df['best_offer'].iloc[i],
           ebay_df['item_condition'].iloc[i],ebay_df['item_price'].iloc[i],ebay_df['item_price_unit'].iloc[i],
            ebay_df['item_sold'].iloc[i],ebay_df['item_title'].iloc[i],
           ebay_df['item_type'].iloc[i],ebay_df['return_policy'].iloc[i],
           ebay_df['seller_name'].iloc[i],ebay_df['seller_score'].iloc[i],ebay_df['shiping_price'].iloc[i],
            ebay_df['shiping_price_compute'].iloc[i])]
    mycursor.executemany(sql, val)
    mydb.commit()
    

f) Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

In [105]:
#f

mycursor.execute('SELECT item_type,item_condition,min(best_offer),\
                avg(best_offer),max(best_offer) ,std(best_offer) FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============Best Offer===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============Best Offer===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', '0', 0.034482758620689655, '1', 0.18246560765962694)
('sponsored', 'Used', '0', 0.18181818181818182, '1', 0.38569460791993504)
('sponsored', 'Manufacturer refurbished', '0', 0.0, '0', 0.0)
('sponsored', 'Open box', '0', 0.0, '0', 0.0)
('non-sponsored', 'New', '0', 0.34782608695652173, '1', 0.4762804847871009)
('non-sponsored', 'Seller refurbished', '0', 0.09836065573770492, '1', 0.2978016741735238)
('non-sponsored', 'Used', '0', 0.4642082429501085, '1', 0.49871730482035737)
('non-sponsored', 'Manufacturer refurbished', '0', 0.08571428571428572, '1', 0.27994168488950605)
('non-sponsored', 'Open box', '0', 0.09473684210526316, '1', 0.2928511103840748)


In [106]:


mycursor.execute('SELECT item_type,item_condition,min(item_price),\
                avg(item_price),max(item_price),std(item_price) FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============PRICE===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============PRICE===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', 8199, Decimal('14363.5172'), 24500, 4032.226001300622)
('sponsored', 'Used', 3999, Decimal('12305.2727'), 24999, 6728.080248831069)
('sponsored', 'Manufacturer refurbished', 7998, Decimal('17517.2118'), 92499, 10875.584796278225)
('sponsored', 'Open box', 19895, Decimal('19895.0000'), 19895, 0.0)
('non-sponsored', 'New', 14999, Decimal('51725.0522'), 189999, 25881.063235952366)
('non-sponsored', 'Seller refurbished', 6998, Decimal('18587.4098'), 99999, 15305.65218577877)
('non-sponsored', 'Used', 3300, Decimal('15836.9414'), 99999, 11628.528379093417)
('non-sponsored', 'Manufacturer refurbished', 4500, Decimal('22638.7786'), 96999, 14446.291987945191)
('non-sponsored', 'Open box', 8995, Decimal('23314.2000'), 82437, 13255.807799324479)


In [107]:
mycursor.execute('SELECT item_type,item_condition,item_price_unit,count(item_price_unit)\
                FROM eBay_items group by item_type,item_condition,item_price_unit')
myresult = mycursor.fetchall()
print('============Item Price Unit===================')
print('Type - Condition - Num')
for x in myresult:
    print(x)

============Item Price Unit===================
Type - Condition - Num
('sponsored', 'Seller refurbished', 'US', 29)
('sponsored', 'Used', 'US', 11)
('sponsored', 'Manufacturer refurbished', 'US', 85)
('sponsored', 'Open box', 'US', 1)
('non-sponsored', 'New', 'US', 115)
('non-sponsored', 'Seller refurbished', 'US', 181)
('non-sponsored', 'Used', 'US', 457)
('non-sponsored', 'Manufacturer refurbished', 'US', 140)
('non-sponsored', 'Open box', 'US', 95)
('non-sponsored', 'Seller refurbished', 'AU', 2)
('non-sponsored', 'Used', 'C', 3)
('non-sponsored', 'Used', 'GBP', 1)


In [108]:
mycursor.execute('SELECT item_type,item_condition,min(item_sold),\
                avg(item_sold),max(item_sold),std(item_sold) FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============ITEM SOLD===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============ITEM SOLD===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', 0, Decimal('160.2069'), 801, 179.98340523318845)
('sponsored', 'Used', 0, Decimal('34.5455'), 148, 49.75277723881559)
('sponsored', 'Manufacturer refurbished', 0, Decimal('139.0824'), 871, 179.64476133975242)
('sponsored', 'Open box', 388, Decimal('388.0000'), 388, 0.0)
('non-sponsored', 'New', 0, Decimal('1.4435'), 17, 3.0848615058697493)
('non-sponsored', 'Seller refurbished', 0, Decimal('71.6393'), 973, 159.37470362936233)
('non-sponsored', 'Used', 0, Decimal('6.7419'), 536, 44.61049267145255)
('non-sponsored', 'Manufacturer refurbished', 0, Decimal('58.1714'), 762, 125.70333231502676)
('non-sponsored', 'Open box', 0, Decimal('37.0421'), 753, 110.51676850995804)


In [109]:
mycursor.execute('SELECT item_type,item_condition,count( item_title)\
                FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============Item Title===================')
print('Type - Condition - Num')
for x in myresult:
    print(x)

============Item Title===================
Type - Condition - Num
('sponsored', 'Seller refurbished', 29)
('sponsored', 'Used', 11)
('sponsored', 'Manufacturer refurbished', 85)
('sponsored', 'Open box', 1)
('non-sponsored', 'New', 115)
('non-sponsored', 'Seller refurbished', 183)
('non-sponsored', 'Used', 461)
('non-sponsored', 'Manufacturer refurbished', 140)
('non-sponsored', 'Open box', 95)


In [110]:
mycursor.execute('with table1 as (SELECT item_type,item_condition,CASE\
    WHEN return_policy ="Seller does not accept returns" THEN 0\
    WHEN return_policy ="" Then NULL\
    ELSE 1 END as return_allow\
                FROM eBay_items) select item_type,item_condition,min(return_allow),avg(return_allow),max(return_allow),std(return_allow)\
                From table1 group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============Return Policy===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============Return Policy===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', 0, Decimal('0.9655'), 1, 0.18246560765962694)
('sponsored', 'Used', 0, Decimal('0.9091'), 1, 0.28747978728803447)
('sponsored', 'Manufacturer refurbished', 1, Decimal('1.0000'), 1, 0.0)
('sponsored', 'Open box', 1, Decimal('1.0000'), 1, 0.0)
('non-sponsored', 'New', 0, Decimal('0.6348'), 1, 0.4814910678228566)
('non-sponsored', 'Seller refurbished', 0, Decimal('0.9344'), 1, 0.2475355552547787)
('non-sponsored', 'Used', 0, Decimal('0.5336'), 1, 0.49886824260758983)
('non-sponsored', 'Manufacturer refurbished', 0, Decimal('0.9214'), 1, 0.26906868859767663)
('non-sponsored', 'Open box', 0, Decimal('0.8737'), 1, 0.3322049229065158)


In [111]:
mycursor.execute('SELECT item_type,item_condition,count(DISTINCT seller_name)\
                FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============Seller===================')
print('Type - Condition - Num')
for x in myresult:
    print(x)

============Seller===================
Type - Condition - Num
('non-sponsored', 'Manufacturer refurbished', 46)
('non-sponsored', 'New', 65)
('non-sponsored', 'Open box', 39)
('non-sponsored', 'Seller refurbished', 60)
('non-sponsored', 'Used', 311)
('sponsored', 'Manufacturer refurbished', 8)
('sponsored', 'Open box', 1)
('sponsored', 'Seller refurbished', 8)
('sponsored', 'Used', 7)


In [112]:
mycursor.execute('SELECT item_type,item_condition,min(seller_score),\
                avg(seller_score),max(seller_score),std(seller_score) FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============SELLER SCORE===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============SELLER SCORE===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', '12391', 54359.793103448275, '84740', 67159.08145455284)
('sponsored', 'Used', '11008', 37919.181818181816, '84740', 59273.82861051545)
('sponsored', 'Manufacturer refurbished', '102008', 84772.98823529412, '84740', 41789.366885818075)
('sponsored', 'Open box', '84740', 84740.0, '84740', 0.0)
('non-sponsored', 'New', '0', 8746.617391304348, '987', 26509.822457405782)
('non-sponsored', 'Seller refurbished', '1053', 181218.0655737705, '9929', 463136.19343829027)
('non-sponsored', 'Used', '0', 19228.26464208243, '9905', 50795.71440716107)
('non-sponsored', 'Manufacturer refurbished', '0', 140093.3142857143, '987', 281105.4696322585)
('non-sponsored', 'Open box', '0', 40451.336842105266, '987', 53824.19300856614)


In [113]:
mycursor.execute('SELECT item_type,item_condition,min(shiping_price),\
                avg(shiping_price),max(shiping_price),std(shiping_price)  FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============SHIPPING PRICE===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============SHIPPING PRICE===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', 0, Decimal('0.0000'), 0, 0.0)
('sponsored', 'Used', 0, Decimal('45.0000'), 495, 142.30249470757707)
('sponsored', 'Manufacturer refurbished', 0, Decimal('0.0000'), 0, 0.0)
('sponsored', 'Open box', 0, Decimal('0.0000'), 0, 0.0)
('non-sponsored', 'New', 0, Decimal('335.3737'), 2998, 736.8133761200772)
('non-sponsored', 'Seller refurbished', 0, Decimal('73.8611'), 2400, 363.3416782510933)
('non-sponsored', 'Used', 0, Decimal('328.9078'), 4995, 618.6144671205246)
('non-sponsored', 'Manufacturer refurbished', 0, Decimal('46.2612'), 4000, 367.8398809894537)
('non-sponsored', 'Open box', 0, Decimal('40.3708'), 1595, 222.22002367440544)


In [114]:
mycursor.execute('SELECT item_type,item_condition,min(shiping_price_compute),\
                avg(shiping_price_compute),max(shiping_price_compute),std(shiping_price_compute)\
                FROM eBay_items group by item_type,item_condition')
myresult = mycursor.fetchall()
print('============Shopping Price Comupute===================')
print('Type - Condition - Min - Avg - Max')
for x in myresult:
    print(x)

============Shopping Price Comupute===================
Type - Condition - Min - Avg - Max
('sponsored', 'Seller refurbished', 0, Decimal('0.0000'), 0, 0.0)
('sponsored', 'Used', 0, Decimal('0.0000'), 0, 0.0)
('sponsored', 'Manufacturer refurbished', 0, Decimal('0.0000'), 0, 0.0)
('sponsored', 'Open box', 0, Decimal('0.0000'), 0, 0.0)
('non-sponsored', 'New', 0, Decimal('0.1391'), 1, 0.34608258681969395)
('non-sponsored', 'Seller refurbished', 0, Decimal('0.0164'), 1, 0.12698306053139072)
('non-sponsored', 'Used', 0, Decimal('0.2364'), 1, 0.4248969907746739)
('non-sponsored', 'Manufacturer refurbished', 0, Decimal('0.0429'), 1, 0.20253495541082608)
('non-sponsored', 'Open box', 0, Decimal('0.0632'), 1, 0.24324673701665986)


In [115]:
mydb.close()

g) Use the stats in (f) and tell in your own words, how sponsored and non-sponsored items appear to be different. Is there a variable/column that can be used to predict the sponsor/non-sponsor items. (You do NOT need to run any model or statistic calculations for this part. Make your judgment only by looking at the stats you printed in (f))

In [ ]:
#g

#Best Offer: According to the data, non-sponsored products are more likey to have best offer available.


#Item Price: For the item price, sponsored products have lower average prices than non-sponsored products 
#at each condition level. 

#Item Price Unit: There are some prices in non-sponsored products are not USD. 

#Item Sold:Sponsored products have more items sold on average than non-sponsored products at every condition level 
#except ‘new’. However, when comparing the maximum item sold, non-sponsored products have more items sold.

#Item Title: For the item title, the number of item was computed. For non-sponsored products, most of them are 
#used products. For sponsored products, most products are manufacturer refurbished products. 

#Return Allowed: Sponsored products are more like to accept returns except for ‘new’ products. 

#Seller Number: The number of distinct sellers was computed. For non-sponsored products, most sellers are selling 
#used products. For sponsored products, people who sell manufacturer refurbished and seller refurbished products 
#are the majority.

#Seller Score: Non-sponsored sellers are more likely to have higher seller scores except the 'open-box'.

#Shipping Price: For the shipping price, sponsored products are much more likely to have free shipping fees

#Compute Shipping Price: For the shipping price that needs to be computed, non-sponsored products are more 
#likely to let customers compute the shipping price. 


#Conclusion: best_off, item_price, item_sold, return_allowed, seller_score, shipping_price, 
#and shipping_price_compute may be good predictors to predict the sponsored/nonsponsored items. 